In [ ]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.plotting import plotting_utils_behavior as putils
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)
public = False

In [ ]:
subj = subject.Subject & {'subject_nickname': 'KS019'}
# get the first date when animal became "trained" and "ready for ephys"
status = putils.get_status(subj)
# get date range and mondays
d = putils.get_date_range(subj)

if d['seven_months_date']:
    status['is_over_seven_months'] = True
    status['seven_months_date'] = d['seven_months_date'].strftime('%Y-%m-%d')
else:
    status['is_over_seven_months'] = False

In [ ]:
# get trial counts and session length to date
session_info = (behavior_ingest.TrialSet *
                acquisition.Session & subj).proj(
    'n_trials', session_date='DATE(session_start_time)',
    session_duration='TIMESTAMPDIFF(MINUTE, \
        session_start_time, session_end_time)').fetch(as_dict=True)
session_info = pd.DataFrame(session_info)
session_info = session_info.where((pd.notnull(session_info)), None)

n_trials = session_info['n_trials'].tolist()
max_trials = max(n_trials)
yrange = [0, max_trials+50]

trial_counts = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['n_trials'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='black',
        line=dict(
            color='white',
            width=1
        )
    ),
    name='trial counts',
    yaxis='y1',
    showlegend=False
)

session_length = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['session_duration'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='red',
        line=dict(
            color='white',
            width=1
        )
    ),
    name='session duration',
    yaxis='y2',
    showlegend=False
)

data = [trial_counts, session_length]

# add monday plots
data = putils.create_monday_plot(data, yrange, d['mondays'])

# add status plots
data = putils.create_status_plot(
    data, yrange, status, public=public)

layout = go.Layout(
    yaxis=dict(
        title='Trial counts',
        range=yrange
    ),
    yaxis2=dict(
        title='Session duration (mins)',
        overlaying='y',
        color='red',
        side='right'
    ),
    xaxis=dict(
        title='Date',
        range=[d['first_date_str'], d['last_date_str']],
        showgrid=False
    ),
    width=700,
    height=400,
    title=dict(
        text='Trial counts and session duration',
        x=0.18,
        y=0.85
    ),
    legend=dict(
        x=1.2,
        y=0.8,
        orientation='v'),
    template=dict(
        layout=dict(
            plot_bgcolor="white"
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [ ]:
f = open("trial_counts_session_duration.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()